# 1. Аналоговая реализация алгоритма Нидлмана-Вунша

## Дано

**seq1**: `GACGAAG`

**seq2**: `ACCAAG`

**Параметры**: Match = 1, Mismatch = -1, Indel = -1 

|       | - | **A**  | **C**  | **C**  | **A**  | **A**  |  **G**  |
|-------|----|----|----|----|----|----|----|
|   -   |  **0** | -1 | -2 | -3 | -4 | -5 | -6 |
| **G** | -1 | **-1** | -2 | -3 | -4 | -5 | -4 |
| **A** | -2 |  **0** | -1 | -2 | -2 | -3 | -4 |
| **C** | -3 | -1 | **1**  |  0 | -1 | -2 | -3 |
| **G** | -4 | -2 | 0  | **0** | -1 | -2 | -1 |
| **A** | -5 | -3 | -1 | -1 |  **1** | 0  | -1 |
| **A** | -6 | -4 | -2 | -2 | 0  | **2**  | -1  |
| **G** | -7 | -5 | -3 | -3 | -1  | 1  | **3**  |

## Результат выравнивания
```
seq2: _ A C C A A G
        | |   | | |
seq1: G A C G A A G
```
**score = 5 - 1 - 1 = 3**

# 2. Реализация на питоне алгоритмов Нидлмана-Вунша и Смита-Вотермана

## Алгоритм Нидлмана-Вунша

In [5]:
import numpy as np

def needleman_wunsch(seq1, seq2, match, mismatch, indel):
    n, m = len(seq1), len(seq2)
    matrix = np.zeros((n+1, m+1), dtype=int)
    
    for i in range(1, n+1):
        matrix[i][0] = matrix[i-1][0] + indel
    for j in range(1, m+1):
        matrix[0][j] = matrix[0][j-1] + indel
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = matrix[i-1][j] + indel
            insert = matrix[i][j-1] + indel
            matrix[i][j] = max(match_score, delete, insert)
    print(matrix)
    align1, align2 = [], []
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and matrix[i][j] == matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch):
            align1.append(seq1[i-1])
            align2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif i > 0 and matrix[i][j] == matrix[i-1][j] + indel:
            align1.append(seq1[i-1])
            align2.append('-')
            i -= 1
        else:
            align1.append('-')
            align2.append(seq2[j-1])
            j -= 1

    return matrix[n][m], ''.join(reversed(align1)), ''.join(reversed(align2))

In [19]:
score, align1, align2 = needleman_wunsch(seq1 = "GACGAAG", seq2 = "ACCAAG", match = 1, mismatch = -1, indel = -1)
print('Выравнивание:')
print(align1)
print(align2)
print(f'Score = {score}')

[[ 0 -1 -2 -3 -4 -5 -6]
 [-1 -1 -2 -3 -4 -5 -4]
 [-2  0 -1 -2 -2 -3 -4]
 [-3 -1  1  0 -1 -2 -3]
 [-4 -2  0  0 -1 -2 -1]
 [-5 -3 -1 -1  1  0 -1]
 [-6 -4 -2 -2  0  2  1]
 [-7 -5 -3 -3 -1  1  3]]
Выравнивание:
GACGAAG
-ACCAAG
Score = 3


## Алгоритм Смита-Вотермана

In [6]:
def smith_waterman(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-1):
    
    m, n = len(seq1), len(seq2)
    
    H = np.zeros((m + 1, n + 1), dtype=int)

    max_score = 0
    max_pos = (0, 0)
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = H[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = H[i - 1, j] + gap_penalty
            insert = H[i, j - 1] + gap_penalty
            H[i, j] = max(match, delete, insert)  
            
            if H[i, j] > max_score:
                max_score = H[i, j]
                max_pos = (i, j)
    
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_pos
    
    while i > 0 and j > 0:
        if H[i, j] == H[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif H[i, j] == H[i - 1, j] + gap_penalty:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
    
    aligned_seq1.reverse()
    aligned_seq2.reverse()
    
    return H, "".join(aligned_seq1), "".join(aligned_seq2), max_score

seq1 = "GACGAAG"
seq2 = "ACCAAG"

H, align1, align2, score = smith_waterman(seq1, seq2)

print("Матрица очков:")
print(H)
print("\nВыравнивание:")
print(align1)
print(align2)
print("\nСчет:", score)


Матрица очков:
[[ 0  0  0  0  0  0  0]
 [ 0 -1 -1 -1 -1 -1  1]
 [ 0  1  0 -1  0  0  0]
 [ 0  0  2  1  0 -1 -1]
 [ 0 -1  1  1  0 -1  0]
 [ 0  1  0  0  2  1  0]
 [ 0  1  0 -1  1  3  2]
 [ 0  0  0 -1  0  2  4]]

Выравнивание:
ACGAAG
ACCAAG

Счет: 4


# 3. Библиотеки Python

In [1]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [7]:
from Bio import pairwise2
alignments = pairwise2.align.globalms("GACGAAG", "ACCAAG", 1, -1, -1, -1)
print(pairwise2.format_alignment(*alignments[0]))

GACGAAG
 ||.|||
-ACCAAG
  Score=3



In [3]:
alignments = pairwise2.align.localms("GACGAAG", "ACCAAG", 1, -1, -1, -1)
print(pairwise2.format_alignment(*alignments[0]))

2 ACGAAG
  ||.|||
1 ACCAAG
  Score=4

